# Multi-hospital Organ Transplants

This notebook explores expectations related to our organ transplant scenario. If you have not yet done so, please refer to **AgentsAndEvents.ipynb** for important events and fluents used in this scenario, and **Expectations.ipynb** for a discussion of expectations in this context. Both of these notebooks are located in the parent directory, **MultipleAgents**.

### Set up

We need to set up our environment by loading in our event and fluent declarations from **dec:notation.pl** and **AE.pl**. The latter contains the code from **AgentsAndEvents.ipynb** the last time that notebook was run.

<font color='red'>IMPORTANT NOTE: you should run **AgentsAndEvents.ipynb** prior to running this notebook to ensure that **AE.pl** contains up to date predicates! You must run the notebook, not just update and save it.</font>

In [ ]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).
?- ['AE'].

true.
false.
true.
true.

## Authorisation

In [ ]:
% File: waitListAdditions.pl
happensAtNarrative("Otago":send(monSys, waitAddReq(101, ["liver", "kidney"], [sex:"F", 
dob:"01/01/2000"])),0).
happensAtNarrative("Otago":send(monSys, waitAddReq(102, ["heart"], [sex:"M", dob:"20/06/1930", 
conditions:["LungCancer"]])),1).
happensAtNarrative("Ashburton":send(monSys, waitAddReq(103, ["kidney"], [sex:"M", dob:"01/10/2022"])),1).
happensAtNarrative("Otago":send(monSys, waitAddReq(104, ["heart", "lung"], [sex:"M",dob:"01/04/2015", 
conditions:["HepC"]])),2).
happensAtNarrative("Otago":send(monSys, waitAddReq(105, ["lung"], [sex:"F", dob:"01/04/2022", 
conditions:["HepC","Asthma"]])),3).

% Requests to remove a patient from the waiting list.
happensAtNarrative("Otago":send(monSys, waitDelReq(101, "Patient Death")),2).

In [ ]:
% File: donorOffers.pl
happensAtNarrative("Wellington": send(monSys, donorOffer(201, [heart, kidney, liver, lungs, pancreas], 
[sex:"M", ethnicity:["Māori"], dob:"10/11/1978", 
conditions:["HepC"]])),2).

In [ ]:
% File: organMatches.pl
happensAtNarrative(monSys:match(401, "Wellington", 201, "Otago", 102, "Auckland", [heart], 
[donorCondition:"HepC"]),4).
happensAtNarrative(monSys:match(402, "Otago", 102, "Christchurch", 16, "Auckland", [kidney, heart], 
[recDisease:"Asthma", patUrgency:3]),4).

In [ ]:
?- run(10).
?- happensAt(Hosp, Event, 0).
?- happensAt(Hosp, Event, 1).
?- causes(A,B, 0).

true.
Hosp = b'Otago', Event = send(monSys, waitAddReq(101, [b'liver', b'kidney'], [Functor(188685,2,sex,b'F'), Functor(188685,2,dob,b'01/01/2000')])) .
Hosp = b'Otago', Event = send(monSys, waitAddReq(102, [b'heart'], [Functor(188685,2,sex,b'M'), Functor(188685,2,dob,b'20/06/1930'), Functor(188685,2,conditions,[b'LungCancer'])])) ;
Hosp = b'Ashburton', Event = send(monSys, waitAddReq(103, [b'kidney'], [Functor(188685,2,sex,b'M'), Functor(188685,2,dob,b'01/10/2022')])) ;
Hosp = monSys, Event = receive(b'Otago', waitAddReq(101, [b'liver', b'kidney'], [Functor(188685,2,sex,b'F'), Functor(188685,2,dob,b'01/01/2000')])) .
A = :(_1620, send(_1626, _1628)), B = :(_1626, receive(_1620, _1628)) ;
A = :(monSys, receive(_1626, waitAddReq(_1632, _1634, _1636))), B = :(monSys, send(_1626, waitAccept(_1632))) ;
A = :(monSys, viol(happ(receive(_1644, waitAddReq(_1650, _1652, _1654))), _1628, _1630, _1632, _1634, _1636)), B = :(monSys, send(_1644, waitReject(_1650, _1636))) ;
A = :(monSys, noMatch(_1

An agent may expect that other agents that it interacts with will be authorised in some manner. Here, we declare that **monSys** expects that waiting list add requests and donation offers come from agents that **monSys** views to be registered hospitals.

We also declare that **monSys** expects that the transplant location specified in a match event is a hospital declared to be known by **monSys**.

In [ ]:
% File: expAuthorisation.pl

% Test expectation rule - hospital wait list requests should come from recognised agent.
initially(monSys:exp_rule(happ(receive(Hospital, waitAddReq(_, _, _))), agent(Hospital), dependent, 
"Authorisation":"Wait list requests should come from recognised hospital")).
initially(monSys:exp_rule(happ(receive(Hospital, donorOffer(PatID, Organs, Details))), agent(Hospital),
 dependent, "Authorisation":"Donation offers should come from a recognised hospital")).


initially(monSys:exp_rule(happ(match(_, _, _, _, _, Location, _, _)), agent(Location), dependent, 
"Authorisation":"Transplant location should be a registered hospital")).

An agent might expect that they are kept up to date on important information which is provided by other agents. In this case, **monSys** expects that the hospital which has added a patient to the organ waiting list will provide updates about the patient's information at least every 5 time periods, up until the patient is removed from the waiting list. 

In reality, the maximum time allowed between patient updates might be 6 months, but for testing purposes, the time period is kept low here.

In [ ]:
% File: expWaitListUpdates.pl

% MonSys expect that the relevant hospital will update a patient-s information or the patient will 
% be removed from the waiting list within 5 time periods of them being added or an update provided.
initially(monSys:exp_rule(or(happ(receive(Hospital, waitAddReq(ID,_,_))), happ(receive(Hospital, 
waitUpdate(ID,_,_)))), within(or([happ(receive(Hospital, waitUpdate(ID,_,_))),
not(waiting(ID, Hospital, _))]),5), dependent, 
"Regular updates":"A hospital will update a waiting recipient's information at least every 5 time periods")).

In [ ]:
% File: donationExpectations.pl
% Donation offer to match expectation
initially(monSys:exp_rule(happ(receive(Hospital, donorOffer(ID, Organs, _))), 
within(or([happ(noMatch(Hospital, ID)), happ(match(_, Hospital, ID, _, _, _, Organs, _))]),1), 
dependent, "Timely response":A donation offer should be matched or refused in the next time period"), _).

In [ ]:
% File: matchAcceptance.pl
initiates(monSys:receive(Hospital, acceptMatch(ID)), matchAccepted(Hospital,ID),_).
initially(monSys:exp_rule(happ(send(DonorHospital, recipientFound(MatchID, _, _, _, _, _))),
within(happ(receive(DonorHospital, acceptMatch(MatchID))),3), dependent, 
"Timely response":"A match offer should be accepted by the donor's hospital within 3 time periods")).
initially(monSys:exp_rule(happ(send(RecipientHospital, donorFound(MatchID, _, _, _, _, _))),
within(happ(receive(RecipientHospital, acceptMatch(MatchID))),3), dependent, 
"Timely response":"A match offer should be accepted by the recipient's hospital within 3 time periods")).
initially(monSys:exp_rule(happ(send(Location, locationSelected(MatchID, _, _, _, _))),
within(happ(receive(Location, acceptMatch(MatchID))),3), dependent, 
"Timely response":"A match offer should be accepted by the transplant location within 3 time periods")).

There are many cases where an agent instructs another agent to perform a given action, and wants to be informed of the outcome. 

Hospitals and the monitoring system both want to receive updates on whether a transplant was successful. Individual hospitals may have expectations about the percentage of organs they donate which lead to successful transplants, or about knowing whether their own patient has been successfully treated. The monitoring system may have the capability to calculate system wide statistics on successful transplants, in which case it would expect to be notified whether transplants were successful.

Below, the expectation rules declared state that **monSys** and the donor and recipient hospitals will be notified of the transplant outcome within 5 time periods of the match being confirmed (for **monSys**) or within 5 time periods of being notified of the match confirmation (for the individual hospitals).

In [ ]:
% File: followUp.pl
initially(monSys:exp_rule(and([happ(confirmationMatch(ID)), role(Location, MatchID, transplant)]),
within(happ(receive(Location, transplantOutcome(ID, _))),5),
dependent,"Outcome notification":"monSys will be notified of transplant outcome within X time periods")).

initially(Agent:exp_rule(and([happ(receive(confirmationNotification(ID))), 
role(Location, MatchID, transplant), self(Self), or([role(Self, matchID, donor), 
role(Self, MatchID, recipient)])]), within(happ(receive(Location, transplantOutcome(ID, _))),5), dependent,
"Outcome notification":"The donor and recipient hospitals should be notified of transplant outcome within X time periods")):- 
Agent \= monSys.

In [ ]:
% File: expIndividualMatch.pl
initially("Otago":exp_rule(happ(receive(monSys, recipientFound(_, _, _, _, _, Notes))), 
condition(member(patUrgency:3, Notes)), dependent, "Individual":Recipients of Otago donations should have urgency level 3")).

In [ ]:
% File: matchResponse.pl
initially(monSys:exp_rule(happ(receive(Hospital, donorOffer(ID, Organs, _))), 
within(or([happ(noMatch(Hospital, ID)), happ(match(_, Hospital, ID, _, _, _, Organs, _))]),1), 
dependent, "Timely response":"A donation offer should be matched or refused in the next time period"), _).

A individual hospital may have unique expectations of what makes an organ match suitable, however, it is also likely that there are common expectations which are shared across all of the hospitals in our system. We declare some simple compatibility rules to hold for all agents (which can be achieved by setting the Agent argument to _).

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32f94018-a4da-40ef-8c9f-8983d73811c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>